In [ ]:

from dotenv import load_dotenv
import os

load_dotenv()

DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
DB_HOST = os.getenv("DB_HOST", "localhost")
DB_PORT = os.getenv("DB_PORT", "5432")


# Imports

In [1]:
from datetime import datetime;

import requests
from bs4 import BeautifulSoup

from datetime import datetime, timedelta

import psycopg2 as pg2


# Safe Float method

In [2]:
# Utility function to handle invalid data
def safe_float(value):
    if value == '#DIV/0!' or value == '' or value is None:
        return 0.0  # or you could return None if you prefer to handle it differently
    try:
        return float(value)
    except ValueError:
        return 0.0  # Fallback to 0.0 in case the value can't be converted

# Date method


In [3]:
def generateDate():
    baseDate=datetime.now().strftime('%d-%b-%Y')
    return baseDate

# Check Week Day

In [4]:
# Define the checkIfWeekDay function
def checkIfWeekDay(date):
    isWeekDay = False
    # Convert the string date to a datetime object and check if it's a weekday
    if datetime.strptime(date, '%d-%b-%Y').weekday() < 5:  # 0-4 are weekdays (Mon-Fri)
        isWeekDay = True
    else:
        isWeekDay = False
    return isWeekDay


# Portal Status

In [5]:

def checkPortalStatus(date):
    
    portalURLStatus = f'https://portal.amfiindia.com/DownloadNAVHistoryReport_Po.aspx?frmdt={date}'
    response = requests.get(portalURLStatus)

    if response.status_code != 200:
        print(f"Failed to fetch data. HTTP Status code: {response.status_code}")
    else:
        content = BeautifulSoup(response.text, 'html.parser')
        content = content.get_text(strip=True)

        if 'No data found on the basis of selected parameters for this report' in content:
            portalURLStatus = portalURLStatus+' '+'Down'
        else:
            portalURLStatus = portalURLStatus+' '+'Up'
    
    return portalURLStatus

# Latest Date in DBTable

In [6]:
def getMaxDate(tableName):
    latestDate=None
    try:
        print('Entered MaxDate Method')
        command=f"select max(date) from {tableName}"
        dbConnection = pg2.connect(user="postgres",password="Itsme@pa1",database="MutualFundData")
        dbCursor = dbConnection.cursor()
        dbCursor.execute(command)
        result = dbCursor.fetchone()
        latestDate = result[0].strftime('%d-%b-%Y') if result else None 
        dbCursor.close()
        dbConnection.close()
        print('Method endded')
    except Exception as e:
        print(f"An error occurred: {e}")
        latestDate=None
    
    return latestDate
        


# Main Block

In [7]:
try:
    # Define first and final dates
    tableName='HistoricalData'

    first_date = (datetime.strptime(getMaxDate(tableName), '%d-%b-%Y') + timedelta(days=1)).strftime('%d-%b-%Y')

    final_date = datetime.now().strftime('%d-%b-%Y')

    schemeType = None
    fundHouseName = None
    schemeCode = None
    schemeName = None
    isinDivPayoutORisinGrowth = None
    isinDivReinvestment = None
    nav = None
    repurchasePrice = None
    salePrice = None
    date = None    
    
    # Convert string dates to datetime objects
    start_date = datetime.strptime(first_date, '%d-%b-%Y')
    end_date = datetime.strptime(final_date, '%d-%b-%Y')

    # Loop through the date range
    current_day = start_date
    while current_day <= end_date:
        
        #print(f'Current_day= {current_day}')
        
        date_str = current_day.strftime('%d-%b-%Y')  # Convert current day to string format
        
        print(f'Current_day= {date_str}')
        
        is_weekday = checkIfWeekDay(date_str)  # Check if the current day is a weekday
        
        if is_weekday == False:
            print(f'{date_str} Date falls under Weekend')
            
        if is_weekday == True:
            
            print(f'{date_str} Date is not Weekend')
            msgURL = checkPortalStatus(date_str)
            # time.sleep(3)
            
            if msgURL.endswith('Down'):
                
                print('Portal is Down')
                # continue
          
            if msgURL.endswith('Up'):
                
                print('Portal is Up extracting scheme data')
                
                msgURL = msgURL.replace('Up', '')
                
                msgURL = msgURL.strip()
                
                realTimeResponse = requests.get(msgURL)
                
                realTimeContent = BeautifulSoup(realTimeResponse.text, 'html.parser')

                realTimeContent = realTimeContent.get_text(strip=True)

                lines = realTimeContent.splitlines()
                
                # print(lines)
                
                i = 0
                record_to_insert=[]
                for line_number, line in enumerate(lines, start=1):
                    i = i + 1
                    #print(f'line number is {line_number} and value is {line}')
                    # Skip the first line (if you want to start from the second line)
                    if line_number == 1:
                        continue
    
                    if 'Schemes' in line and line.strip():
                        schemeType = line.strip()
                           
                    # print(schemeType)
                    if 'Mutual Fund' in line and line.strip():
                        fundHouseName = line.strip()
                    # print(fundHouseName)
                    if ';' in line and line.strip():
                        parts = line.split(';')
                        schemeCode = int(parts[0].strip())
                        schemeName = parts[1].strip()
                        isinDivPayoutORisinGrowth = parts[2].strip()
                        isinDivReinvestment = parts[3].strip()
                        nav = safe_float(parts[4].strip())
                        repurchasePrice = safe_float(parts[5].strip()) if parts[5].strip() else 0.0
                        salePrice = safe_float(parts[6].strip()) if parts[6].strip() else 0.0
                        date = datetime.strptime(parts[7].strip(), "%d-%b-%Y").strftime("%d-%m-%Y")
                
                    if len(line) > 0 and ';' in line:
                        #print('reached final if')
                        #print(f"{schemeType} {fundHouseName} {schemeCode} {schemeName} {isinDivPayoutORisinGrowth} {isinDivReinvestment} {nav} {repurchasePrice} {salePrice} {date}")
                        record_to_insert.append((schemeType, fundHouseName, schemeCode, schemeName, isinDivPayoutORisinGrowth, isinDivReinvestment, nav, repurchasePrice, salePrice, date))
                        with open(f"C:\\Users\\B.V.PAVAN KUMAR\\Desktop\\DataAnalyst\\Sample Projects\\MF_Scheme_Data\\OldData\\OP_{date_str}.csv", "a", encoding="utf-8") as f:
                            f.write(f"{schemeType},{fundHouseName},{schemeCode},{schemeName},{isinDivPayoutORisinGrowth},{isinDivReinvestment},{nav},{repurchasePrice},{salePrice},{date}\n")
                    
                connection = pg2.connect(user="postgres",password="Itsme@pa1",database="MutualFundData")
                cursor = connection.cursor()
                #postgres_insert_query = """ INSERT INTO HistoricalData (schemeType,fundHouseName,schemeCode,schemeName,isinDivPayoutORisinGrowth,isinDivReinvestment,nav,repurchasePrice,salePrice,date) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
                postgres_insert_query = f"""INSERT INTO {tableName} (schemeType, fundHouseName, schemeCode, schemeName, isinDivPayoutORisinGrowth, isinDivReinvestment, nav, repurchasePrice, salePrice, date) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
                cursor.executemany(postgres_insert_query, record_to_insert)
                connection.commit()          
                print(f'Extraction Completed for {date_str}')
        
        current_day += timedelta(days=1)  # Increment the day by 1

except Exception as error:
    print("Error:", error)

print('Program Closed')   

Entered MaxDate Method
Method endded
Current_day= 11-Apr-2025
11-Apr-2025 Date is not Weekend
Portal is Up extracting scheme data
Extraction Completed for 11-Apr-2025
Current_day= 12-Apr-2025
12-Apr-2025 Date falls under Weekend
Program Closed
